# Bunch comparison 

This notebook compares the evolution of multiple bunches.

In [2]:
import os
from pprint import pprint
import sys

from ipywidgets import interact
from ipywidgets import widgets
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import proplot as pplt
from scipy import interpolate
from tqdm.notebook import tqdm

sys.path.append("/home/46h/repo/psdist/")
import psdist as ps
import psdist.visualization as psv

In [3]:
pplt.rc["cmap.discrete"] = False
pplt.rc["cmap.sequential"] = "viridis"
pplt.rc["colorbar.width"] = "1.2em"
pplt.rc["cycle"] = "538"
pplt.rc["figure.facecolor"] = "white"
pplt.rc["grid"] = False

## Setup

In [4]:
save = False  # whether to save figures

In [5]:
folder = "../../data/"
sorted(os.listdir(folder))

FileNotFoundError: [Errno 2] No such file or directory: '../data/'

Label each timestamp (simulation).

In [ ]:
timestamps = {
    "corr": "230809033954",
    "decorr": "230809034305",
}

labels = list(timestamps.keys())
prefixes = dict()
for key in timestamps:
    prefix = f"{timestamps[key]}-sim"
    prefixes[key] = prefix

Create output directory.

In [ ]:
outdir = "../figures/compare_{}".format("-".join(timestamps.values()))
outdir

In [ ]:
if not os.path.isdir(outdir):
    os.makedirs(outdir)

def save_figure(filename, **kws):
    if save:
        filename = os.path.join(outdir, filename)
        plt.savefig(filename, **kws)

## Scalar history

Load node positions.

In [ ]:
file = open(os.path.join(folder, f"{prefixes[labels[0]]}_lattice_nodes.txt"), "r")
nodes_dict = dict()
nodes_list = list()
last = offset = 0.0
for i, line in enumerate(file):
    if i == 0:
        continue
    name, position, length = line.rstrip().split()
    position = float(position)
    length = float(length)
    if position < last:
        offset = offset + last
    last = position
    position = position + offset
    nodes_dict[name] = [position, length]
    nodes_list.append([name, position, length])
file.close()

In [ ]:
nodes_list[:10]

Load scalar history.

In [ ]:
histories = dict()
for label, prefix in prefixes.items():
    filename = os.path.join(folder, f"{prefix}_history.dat")
    history = pd.read_csv(filename)
    for index in range(history.shape[0]):
        Sigma = np.zeros((6, 6))
        for i in range(6):
            for j in range(i + 1):
                Sigma[i, j] = Sigma[j, i] = history.loc[index, f"cov_{j}-{i}"]
        history.loc[index, "eps_x"] = np.sqrt(np.linalg.det(Sigma[0:2, 0:2]))
        history.loc[index, "eps_y"] = np.sqrt(np.linalg.det(Sigma[2:4, 2:4]))
        history.loc[index, "eps_z"] = np.sqrt(np.linalg.det(Sigma[4:6, 4:6]))
        history.loc[index, "eps_xy"] = np.sqrt(np.linalg.det(Sigma[0:4, 0:4]))
        history.loc[index, "eps_xyz"] = np.sqrt(np.linalg.det(Sigma[:, :]))    
    if ("mean_" in history.columns) and ("mean_3" not in history.columns):  # ?
        history.rename(columns={"mean_": "mean_3"}, inplace=True)
    history["phi_max"] = -history["z_to_phase_coeff"] * history["z_max"]
    histories[label] = history.copy(deep=True)
sorted(history.columns)

### First-order moments

In [ ]:
linestyles = ["-", "-"]
colors = ["blue4", "red4"]

In [ ]:
_dims = ["x", "xp", "y", "yp", "z", "w"]
_units = ["mm", "mrad", "mm", "mrad", "mm", "MeV"]
for i in range(6):
    fig, ax = pplt.subplots(figsize=(5.0, 1.5))
    for label, ls, c in zip(labels, linestyles, colors):
        ax.plot(
            histories[label]["position"],
            1000.0 * histories[label][f"mean_{i}"],
            ls=ls,
            color=c,
            label=label,
        )
    ax.format(xlabel="Position [m]", ylabel=f"<{_dims[i]}> [{_units[i]}]")
    ax.legend(loc="r", ncols=1, framealpha=0.0)
    save_figure(f"mean_{i}")
    plt.show()

### Second-order moments

In [ ]:
for i in range(6):
    for j in range(i + 1):
        fig, ax = pplt.subplots(figsize=(5.0, 1.5))
        for label, ls, c in zip(labels, linestyles, colors):
            ax.plot(
                histories[label]["position"],
                1.0e6 * histories[label][f"cov_{j}-{i}"],
                ls=ls,
                color=c,
                label=label,
            )
        ax.format(xlabel="Position [m]")
        ax.legend(loc="r", ncols=1, framealpha=0.0)
        save_figure(f"cov_{j}-{i}")
        plt.show()

### Standard deviations

In [ ]:
for i in range(6):
    fig, ax = pplt.subplots(figsize=(5.0, 1.5))
    for label, ls, c in zip(labels, linestyles, colors):
        ax.plot(
            histories[label]["position"],
            1.0e3 * np.sqrt(histories[label][f"cov_{i}-{i}"]),
            ls=ls,
            color=c,
            label=label,
        )
    dims = ["x", "xp", "y", "yp", "z", "dE"]
    units = ["mm", "mrad", "mm", "mrad", "mm", "GeV"]
    ax.format(xlabel="Position [m]", ylabel=f"{dims[i]} rms [{units[i]}]")
    ax.legend(loc="r", ncols=1, framealpha=0.0)
    save_figure(f"rms_{i}")
    plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(5.0, 1.5))
for label, ls, c in zip(labels, linestyles, colors):
    ax.plot(
        histories[label]["position"],
        histories[label]["z_rms_deg"],
        ls=ls,
        color=c,
        label=label,
    )
ax.format(xlabel="Position [m]", ylabel="z rms [deg]")
ax.legend(loc="r", ncols=1, framealpha=0.0)
save_figure(f"rms_4_deg")
plt.show()

### Emittances 

In [ ]:
for tag in ["x", "y", "z", "xy", "xyz"]:
    factor = 1.0
    if tag in ["x", "y", "z"]:  # 2D emittance
        factor = 1.00e+06
    if tag == "xy":  # 4D emittance
        factor = 1.00e+12
    elif tag == "xyz":  # 6D emittance
        factor = 1.00e+18

    fig, ax = pplt.subplots(figsize=(5.0, 1.75))
    for label, ls, c in zip(labels, linestyles, colors):
        ax.plot(
            histories[label]["position"],
            factor * histories[label][f"eps_{tag}"],
            ls=ls,
            color=c,
            label=label,
    )
    ax.legend(loc="r", ncols=1, framealpha=0.0)
    ax.format(xlabel="Position [m]", ylabel="eps_{}".format(tag))
    save_figure(f"eps_{tag}")
    plt.show()

### Extrema

In [ ]:
for dim, unit in zip(["x", "y", "phi"], ["mm", "mm", "deg"]):
    factor = 1.0
    if dim in "xy":
        factor = 1000.0

    fig, ax = pplt.subplots(figsize=(5.0, 1.5))
    for i, label in enumerate(labels):
        history = histories[label]
        history["phi_rms"] = history["z_rms_deg"]
        ax.plot(
            history["position"],
            factor * history[f"{dim}_max"],
            color=colors[i],
            ls=":",
            label=f"max ({label})",
        )
        ax.plot(
            history["position"],
            factor * history[f"{dim}_rms"],
            color=colors[i],
            label=f"rms ({label})",
        )
    ax.legend(loc="r", ncols=1, framealpha=0.0)
    ax.format(xlabel="Position [m]", ylabel=f"{dim} [{unit}]")
    save_figure(f"extrema_rms_{dim}")
    plt.show()

## Phase space distribution 

### Load bunches 

Collect bunch filenames.

In [ ]:
bunch_filenames = dict()
for key in timestamps:
    _filenames = os.listdir(folder)
    _filenames = [f for f in _filenames if timestamps[key] in f]
    _filenames = [f for f in _filenames if "bunch" in f]
    _filenames = [f for f in _filenames if "lostbunch" not in f]
    _filenames = [f for f in _filenames if "smallbunch" not in f]
    _filenames = sorted(_filenames)
    _filenames = [os.path.join(folder, f) for f in _filenames]
    bunch_filenames[key] = _filenames
    n_frames = len(_filenames)
    
for key in timestamps:
    print(key)
    pprint(bunch_filenames[key])

For each bunch, find its index in the history array. This tells us the position, node, energy, etc. for each bunch. We can then convert z to phi if we want.

In [ ]:
bunch_history_indices = dict()
for label in labels:
    bunch_history_indices[label] = []
    history = histories[label]
    for frame, filename in enumerate(bunch_filenames[label]):
        node = filename.split("bunch_")[-1].split(".dat")[0]
        node = node[(node.find("_") + 1):]
        if node == "START":
            index = 0
        elif node == "STOP":
            index = history.shape[0] - 1
        else:
            index = history["node"].tolist().index(node)
        bunch_history_indices[label].append(index)
        print(f"label={label}, frame={frame}, node={node}, history_index={index}")

Load the bunches.

In [ ]:
bunches = dict()
columns = ["x", "xp", "y", "yp", "z", "w", "index", "x0", "xp0", "y0", "yp0", "z0", "w0"]
for label in labels:
    bunches[label] = []
    history = histories[label]
    for filename in tqdm(bunch_filenames[label]):
        bunch = np.loadtxt(filename, comments="%")
        # Convert [m, rad, m, rad, m, GeV] -> [mm, mrad, mm, mrad, mm, MeV]
        bunch[:, :6] *= 1.00e+03
        if bunch.shape[1] >= 13:
            bunch[:, 7:13] *= 1.00e+03
        # Create DataFrame.
        bunch = pd.DataFrame(bunch, columns=columns[:bunch.shape[1]])
        if "index" in bunch.columns:
            bunch["index"] = bunch["index"].astype(int)
        # Convert z --> phi.
        j = bunch_history_indices[label][frame]
        z_to_phase_coeff = history.loc[j, "z_to_phase_coeff"]
        bunch["phi"] = -z_to_phase_coeff * 0.001 * bunch["z"]
        if "z0" in bunch:
            bunch["phi0"] = -z_to_phase_coeff * 0.001 * bunch["z0"]
        bunches[label].append(bunch)

### 1D projections

Decide on units.

In [ ]:
use_phi = True
dims = ["x", "xp", "y", "yp", "z", "w"]
units = ["mm", "mrad", "mm", "mrad", "mm", "MeV"]
if use_phi:
    dims[4] = "phi"
    units[4] = "deg"
dims_units = [f"{dim} [{unit}]" for dim, unit in zip(dims, units)]

#### Partial projections [INT]

In [ ]:
psv.cloud.proj1d_interactive_slice(
    data=[
        [bunch.loc[:, dims].values for bunch in bunches[label]] 
        for label in labels
    ],
    dims=dims,
    units=units,
    slice_type="int",
    options=dict(
        alpha=True,
        auto_plot_res=False,
        kind=True,
        log=True,
        normalize=True,
        scale=True,
    ),
    autolim_kws=dict(pad=0.1),
    share_limits=False,
    update_limits_on_slice=True,
    legend=True,
    labels=labels,
    colors=colors,
)

#### Full projections

The following cell saves all 1D projections. 

In [ ]:
if save:
    for frame in trange(n_frames):
        data = [
            ps.cloud.norm_xxp_yyp_zzp(
                bunches[label][frame].loc[:, dims].values,
                scale_emittance=True
            )
            for label in labels
        ]
        for axis in range(6):
            profiles, edges = [], []
            ymax = -np.inf
            for X in data:
                points = X[:, axis]
                profile, _edges = np.histogram(points / np.std(points), bins=85, density=True)
                ymax = max(ymax, np.max(profile))
                profiles.append(profile)
                edges.append(_edges)
    
            fig, ax = pplt.subplots(figwidth=3.5, figheight=2.75)
            for i, label in enumerate(labels):
                psv.plot_profile(
                    profile=(profiles[i] / ymax),
                    edges=edges[i],
                    ax=ax,
                    color=colors[i], 
                    label=label,
                    kind="step",
                    scale=None,
                    fill=False,
                    lw=1.5,
                )
            ax.format(yscale="log", yformatter="log")
            ax.format(xlabel=r"{} / $\sigma$".format(dims[axis]))
            ax.legend(loc="right", framealpha=0.0, ncols=1)
            save_figure(f"proj1d_{axis}_{frame}")
            plt.close()

### 2D projections

#### Partial projections [INT]

In [ ]:
psv.cloud.proj2d_interactive_slice(
    data=[
        [bunch.loc[:, dims].values for bunch in bunches[label]] 
        for label in labels
    ],
    dims=dims,
    units=units,
    autolim_kws=dict(pad=0.1, zero_center=True, sigma=None, share=None),
    options=dict(
        log=True,
        mask=True,
        normalize=True,
        ellipse=True,
        profiles=True,
    ),
    share_limits=1,
    process_kws=dict(norm="max"),
    colorbar=True,
    colorbar_kw=dict(tickminor=True),
    fig_kws=dict(toplabels=labels),
    cmap=pplt.Colormap("Blues", left=0.05),
)

#### Full projections (side-by-side)

In [ ]:
if save:
    for frame in trange(n_frames):
        data = [
            ps.cloud.norm_xxp_yyp_zzp(
                bunches[label][frame].loc[:, dims].values, scale_emittance=True
            )
            for label in labels
        ]
        limits = [psv.cloud.auto_limits(X, zero_center=True, pad=0.1) for X in data]
        limits = psv.combine_limits(limits)
        for i in range(6):
            for j in range(i):
                axis = (j, i)
                fig, axs = pplt.subplots(ncols=2)
                for ax, label, X in zip(axs, labels, data):
                    psv.cloud.plot2d(
                        X[:, axis],
                        ax=ax,
                        kind="hist",
                        bins=75,
                        limits=[limits[axis[0]], limits[axis[1]]],
                        process_kws=dict(norm="max"),
                        offset=1.0,
                        norm="log",
                        rms_ellipse=True,
                        rms_ellipse_kws=dict(
                            level=[1.0, 2.0, 3.0, 4.0, 5.0],
                            color="white",
                            alpha=0.2,
                            lw=0.4,
                        ),
                        colorbar=True,
                        colorbar_kw=dict(width="1.2em", tickminor=True),
                        cmap=pplt.Colormap("viridis"),
                    )
                axs.format(xlabel=dims[axis[0]], ylabel=dims[axis[1]], toplabels=labels)
                axs.format(xlim=limits[axis[0]], ylim=limits[axis[1]])
                save_figure(f"proj2d_{axis[0]}-{axis[1]}_{frame}")
                plt.close()

#### Full projections (joint plot, overlay) [INT]

In [ ]:
@interact(
    frame=widgets.BoundedIntText(min=0, max=len(bunches[labels[0]])),
    dim1=widgets.Dropdown(options=dims, value=dims[0]),
    dim2=widgets.Dropdown(options=dims, value=dims[1]),
    logvmin=widgets.FloatSlider(min=-7.0, max=0.0, value=-5.0, continuous_update=False),
    blur_sigma=widgets.FloatSlider(min=0.0, max=4.0, value=0.0),
    nlines=widgets.BoundedIntText(min=3, max=15, value=6),
    bins=widgets.BoundedIntText(min=2, max=512, value=64),
    log=True,
    normalize=True,
    alpha=widgets.FloatSlider(min=0.0, max=1.0, value=1.0),
)
def update_joint_overlay(
    frame, dim1, dim2, logvmin, blur_sigma, nlines, bins, log, normalize, alpha
):
    if dim1 == dim2:
        return
    axis = [dims.index(dim1), dims.index(dim2)]

    data = [bunches[label][frame].iloc[:, axis].values for label in labels]
    if normalize:
        data = [ps.cloud.norm_xxp_yyp_zzp(X, scale_emittance=True) for X in data]

    vmin = 10.0**logvmin
    if log:
        levels = 10.0 ** np.linspace(logvmin, 1.0, nlines, endpoint=False)
    else:
        levels = np.linspace(vmin, 1.0, nlines, endpoint=False)

    grid = psv.JointGrid(
        marg_kws=dict(space="2.0em", width="10.0em"),
        marg_fmt_kws_x=dict(yspineloc="left", xspineloc="bottom"),
        marg_fmt_kws_y=dict(yspineloc="left", xspineloc="bottom"),
        xspineloc="bottom",
        yspineloc="left",
    )
    for i, (label, X) in enumerate(zip(labels, data)):
        grid.plot_cloud(
            X,
            kind="contour",
            levels=levels,
            bins=bins,
            limits=psv.cloud.auto_limits(X, pad=0.1),
            marg_kws=dict(kind="step", fill=False, color=colors[i], lw=1.5, alpha=alpha),
            marg_hist_kws=dict(bins=bins),
            lw=1.5,
            colors=colors[i],
            process_kws=dict(norm="max", blur_sigma=blur_sigma),
            alpha=alpha,
        )
    # Format panel axes.
    ymin = 10.0**-6.0
    ymax = 2.0
    grid.ax_marg_x.format(yformatter="log", yscale="log", ymin=ymin, ymax=ymax)
    grid.ax_marg_y.format(xformatter="log", xscale="log", xmin=ymin, xmax=ymax)
    grid.ax.format(xlabel=dim1, ylabel=dim2)
    return ax

#### Full projections (joint plot, overlay) [SAVE]

In [ ]:
if save:
    for frame in trange(n_frames):
        # for i in trange(6):
        #     for j in range(i):
        for (i, j) in [(1, 0), (3, 2), (2, 0)]:
            ax = update_joint_overlay(
                frame=frame,
                dim1=dims[j],
                dim2=dims[i],
                logvmin=-5.0,
                blur_sigma=1.0,
                nlines=5,
                bins=64,
                log=True,
                normalize=True,
                alpha=0.9,
            )
            save_figure(f"proj2d_joint_{j}-{i}_{frame}.png", dpi=350)
            plt.close()

#### Corner plot (overlay)

In [ ]:
if save:
    blur_sigma = 1.0
    logvmin = -5.0
    nlines = 5
    bins = 64
    alpha = 0.9
    lw = 1.2

    for frame, X in enumerate(tqdm(bunches[label])):
        data = [bunches[label][frame].loc[:, dims].values for label in labels]
        grid = psv.CornerGrid(d=6, corner=True, diag_rspine=True, space=1.4, labels=dims)
        for i, (label, X) in enumerate(zip(labels, data)):
            grid.plot_cloud(
                ps.cloud.norm_xxp_yyp_zzp(X, scale_emittance=True),
                lower=True, 
                upper=False,     
                autolim_kws=dict(pad=0.1, zero_center=True),
                diag_kws=dict(kind="step", fill=False, color=colors[i], lw=lw, alpha=alpha),
                process_kws=dict(norm="max", blur_sigma=blur_sigma),
                kind="contour",
                bins=bins,
                levels=(10.0 ** np.linspace(logvmin, 1.0, nlines, endpoint=False)),   
                colors=colors[i],
                alpha=alpha,
                lw=lw,                        
            )
        grid.format_diag(
            yscale="log", 
            yformatter="log", 
            ymin=1.00e-06,
        )
        save_figure(f"corner_overlay_{frame}")
        plt.close()

### Radial density

Here we compute the density within spherical shells.

#### Overaly [INT]

In [ ]:
@interact(
    frame=widgets.BoundedIntText(min=0, max=(len(bunches[labels[0]]) - 1), value=0),
    bins=(25, 75),
    log=True,
    x=True,
    px=True,
    y=False,
    py=False,
    z=False,
    pz=False,
)
def update_radial_density(frame, bins, log, x, px, y, py, z, pz):
    _bunches = [
        ps.cloud.norm_xxp_yyp_zzp(bunches[label][frame].loc[:, dims].values, scale_emittance=True) 
        for label in labels
    ]
    axis = tuple([i for i, check in enumerate([x, px, y, py, z, pz]) if check])

    fig, ax = pplt.subplots()
    for label, _X in zip(labels, _bunches):
        hist_r, edges_r = ps.cloud.radial_histogram(_X[:, axis], bins=bins, centers=False)
        hist_r = hist_r / np.max(hist_r)
        centers_r = ps.utils.centers_from_edges(edges_r)
        ax.plot(centers_r, hist_r, drawstyle="steps-mid", label=label)
    ax.format(
        xmin=0.0,
        xlabel="radius", ylabel="density",
        title=f"axis={axis}", title_kw=dict(fontsize="medium"),
    )

    ymin = min(hist_r[hist_r > 0])
    ax.plot(centers_r, np.exp(-0.5 * (centers_r**2)), color="black", alpha=0.2)
    ax.format(ymin=ymin)

    if log:
        ax.format(yscale="log", yformatter="log")
    ax.legend(loc="upper right", ncols=1)
    return ax

#### Overlay [SAVE]

In [ ]:
axis_list_all = []
for i in range(6):
    for j in range(i):
        axis_list_all.append((j, i))
        for k in range(j):
            axis_list_all.append((k, j, i))
            for l in range(k):
                axis_list_all.append((l, k, j, i))
                for m in range(l):
                    axis_list_all.append((m, l, k, j, i))
axis_list_all.append((0, 1, 2, 3, 4, 5))

In [ ]:
# axis_list = axis_list_all
axis_list = [
    (0, 1),
    (2, 3),
    (4, 5),
    (0, 1, 2, 3),
    (0, 1, 4, 5),
    (2, 3, 4, 5),
    (0, 1, 2, 3, 4, 5),
]

In [ ]:
if save:
    for frame in range(n_frames):
        for axis in tqdm(axis_list):
            switches = 6 * [False]
            for k in axis:
                switches[k] = True
            x, xp, y, yp, z, pz = switches
            ax = update_radial_density(
                frame=frame,
                bins=50,
                log=True,
                x=switches[0],
                px=switches[1],
                y=switches[2],
                py=switches[3],
                z=switches[4],
                pz=switches[5],
            )

            _dims = [r"x", r"x'", "y", "y'", "z", "z'"]
            xlabel = "-".join([_dims[k] for k in axis])
            xlabel = f"Radius ({xlabel})"
            ax.format(yscale="log", yformatter="log", xlabel=xlabel, ylabel="Density", xmin=0.0)
            ax.legend(loc="upper right", ncols=1)

            tag = "-".join([str(k) for k in axis])
            save_figure(f"radial_{tag}_{frame}")
            plt.close()

## Losses

### Scalar 

NOTE: Apertures here correspond to one MPI node! Fix this!

In [ ]:
apertures = dict()
for key in prefixes:
    apertures[key] = pd.read_csv(os.path.join(folder, f"{prefixes[key]}_losses.txt"), sep=" ")
    
apertures[labels[0]].head()

#### Cumulative loss

This plot is for all MPI nodes; it is correct.

In [ ]:
fig, ax = pplt.subplots(figsize=(5.0, 2.0))
for label, ls, c in zip(labels, linestyles, colors):
    ax.plot(
        histories[label]["position"],
        abs(histories[label]["n_parts"] - histories[label].loc[0, "n_parts"]),
        label=label,
        ls=ls,
        color=c,
    )
ax.legend(loc="r", ncols=1, framealpha=0.0)
ax.format(xlabel="Position [m]", ylabel="Loss (cumulative)")
save_figure("loss_cumulative")
plt.show()

#### Transverse/longitudinal loss

In [ ]:
fig, ax = pplt.subplots(figsize=(4.0, 2.0))
plot_kws = dict(lw=0, marker=".", ms=3.0)
for i, label in enumerate(labels):
    ax.plot(
        apertures[label]["position"].values[i:],
        apertures[label]["loss"].values[i:],
        color=colors[i],
        ls=linestyles[i],
        label=label,
        **plot_kws
    )
ax.legend(loc="ur", ncols=1, framealpha=1.0)
ax.format(xlabel="Position [m]", ylabel="Loss")
save_figure(f"loss_aperture")

Isolate transverse/longitudinal losses.

In [ ]:
fig, axs = pplt.subplots(nrows=2, figsize=(5.0, 3.0), spany=False, aligny=True)
for label in labels:
    _apertures = apertures[label]
    idx_trans, idx_long, idx_long_phase, idx_long_energy = [], [], [], []
    for i in range(_apertures.shape[0]):
        node = _apertures.loc[i, "node"]
        loss = _apertures.loc[i, "loss"]
        if "phase_aprt" in node:
            idx_long_phase.append(i)
            idx_long.append(i)
        elif "energy_aprt" in node:
            idx_long_energy.append(i)
            idx_long.append(i)
        else:
            idx_trans.append(i)
    for ax, idx in zip(axs, [idx_trans, idx_long]):
        ax.plot(
            _apertures["position"].values[idx],
            _apertures["loss"].values[idx],
            label=label,
            **plot_kws
        )
axs[0].legend(loc="r", ncols=1, framealpha=0.0)
axs[0].format(ylabel="Loss (trans)")
axs[1].format(ylabel="Loss (long)", xlabel="Position [m]")
save_figure("loss_aperture_rz")

Now isolate phase/energy apertures.

In [ ]:
fig, axs = pplt.subplots(nrows=2, figsize=(5.0, 3.0), spany=False, aligny=True)
for label in labels:
    _apertures = apertures[label]
    idx_long_phase, idx_long_energy = [], []
    for i in range(_apertures.shape[0]):
        node = _apertures.loc[i, "node"]
        loss = _apertures.loc[i, "loss"]
        if "phase_aprt" in node:
            idx_long_phase.append(i)
        elif "energy_aprt" in node:
            idx_long_energy.append(i)
    for ax, idx in zip(axs, [idx_long_phase, idx_long_energy]):
        ax.plot(
            _apertures["position"].values[idx],
            _apertures["loss"].values[idx],
            label=label,
            **plot_kws
        )
axs[0].legend(loc="r", ncols=1, framealpha=0.0)
axs[0].format(ylabel="Loss (phase)")
axs[1].format(ylabel="Loss (energy)", xlabel="Position [m]")
save_figure("loss_aperture_phase_energy")

#### Aperture node positions

In [ ]:
label = labels[0]
_apertures = apertures[label]
idx_long_phase, idx_long_energy, idx_trans = [], [], []
for i in range(_apertures.shape[0]):
    node = _apertures.loc[i, "node"]
    loss = _apertures.loc[i, "loss"]
    if "phase_aprt" in node:
        idx_long_phase.append(i)
    elif "energy_aprt" in node:
        idx_long_energy.append(i)
    else:
        idx_trans.append(i)

fig, axs = pplt.subplots(nrows=2, figsize=(8.0, 1.5))
for ax, idx in zip(axs, [idx_trans, idx_long_phase]):
    for i in idx:
        ax.axvline(_apertures.loc[i, "position"], lw=0.7, color="black")
axs.format(
    xlabel="Position [m]", 
    yticks=[],
    leftlabels=["trans", "long"], 
    leftlabels_kw=dict(rotation=0, fontweight="normal", fontsize="med"),
    xmin=-0.25, xmax=10.0,
)

# Plot red lines for MEBT RF cavities.
for i, (node, position, length) in enumerate(nodes_list):
    if node in [f"MEBT_RF:Bnch0{j}:Rg01" for j in range(1, 5)]:
        ax.axvline(position, color="red")
save_figure("loss_aperture_locations")

### Phase space distribution

Get lost bunch filenames.

In [ ]:
filenames = dict()
for label in labels:
    _filenames = [f for f in os.listdir(folder)] 
    _filenames = [f for f in _filenames if timestamps[label] in f]
    _filenames = [f for f in _filenames if "lostbunch" in f]
    _filenames = [os.path.join(folder, f) for f in _filenames]
    filenames[label] = _filenames[0]
filenames

Load lost bunches.

In [ ]:
lostbunches = dict()
for label in labels:
    lostbunch = np.loadtxt(filenames[label], comments="%")
    lostbunch[:, :6] *= 1000.0    
    lostbunch = pd.DataFrame(lostbunch, columns=["x", "xp", "y", "yp", "z", "dE", "s", "index"])
    lostbunch["index"] = lostbunch["index"].astype(int)
    lostbunch = lostbunch.sort_values("s")
    z_to_phase_interp = interpolate.interp1d(
        histories[label]["position"], histories[label]["z_to_phase_coeff"], kind="linear"
    )
    lostbunch["phi"] = -z_to_phase_interp(lostbunch["s"]) * 0.001 * lostbunch["z"]
    lostbunches[label] = lostbunch
    
lostbunches[labels[0]]

Check lost particle positions against number of surviving particles.

In [ ]:
for label in labels:
    history = histories[label]
    fig, ax = pplt.subplots(figsize=(5.0, 1.7))
    ax.plot(history["position"], abs(history["n_parts"] - history.loc[0, "n_parts"]), color="black")
    ax.format(xlabel="Position [m]", ylabel="Loss (cumulative)")
    for s in lostbunch["s"].values:
        ax.axvline(s, color="red2")
    save_figure("loss_check_positions")
    print(label)
    plt.show()

#### Lost r-z coordinates

In [ ]:
for label in labels:
    lostbunch = lostbunches[label]
    lostbunch["r"] = np.sqrt(lostbunch["x"]**2 + lostbunch["y"]**2)

fig, axs = pplt.subplots(ncols=2)
for ax, label in zip(axs, labels):
    lostbunch = lostbunches[label]
    ax.scatter(
        lostbunch["phi"], 
        lostbunch["r"], 
        c=lostbunch["s"], 
        cmap=pplt.Colormap("blues", left=0.1), 
        s=8.0,
        colorbar=True,
        colorbar_kw=dict(width="1.1em", label="s [m]"),
    )
    ax.format(xlabel=r"phi [deg]", ylabel="radius [mm]", title=label)
save_figure("loss_rz")

#### Transverse/longitudinal 

Group lost particles into "small z" and "large z" groups to isolate transverse vs. longitudinal losses.

In [ ]:
phi_max = 89.0  # [deg]
lost_index = dict()
for label in labels:
    lostbunch = lostbunches[label]
    index = lostbunch["index"].values
    abs_phi = np.abs(lostbunch["phi"].values)
    lost_index[label] = {
        "all": index,
        "transverse": index[abs_phi <= phi_max],
        "longitudinal": index[abs_phi > phi_max],
    }

Plot loss position histogram.

In [ ]:
smin = 0.0  # start at lattice entrance
smax = nodes_list[-1][1]  # stop at last node
bins = np.linspace(smin, smax, 50)

for log in [False, True]:
    fig, axs = pplt.subplots(
        figsize=(5.0, 2.75),
        nrows=3,
        spany=False,
        aligny=True,
        height_ratios=[0.5, 1.0, 1.0],
        hspace=[0.0, None],
    )
    for label, ls, c in zip(labels, linestyles, ["blue5", "pink5"]):
        lostbunch = lostbunches[label]
        for row, ax in enumerate(axs[1:]):
            if row == 0:
                idx = np.abs(lostbunch["phi"].values) <= phi_max
                ax.format(ylabel="Loss (xy)")
            else:
                idx = np.abs(lostbunch["phi"].values) > phi_max
                ax.format(ylabel="Loss (z)")
            ax.hist(
                lostbunch.loc[idx, "s"].values,
                bins=bins,
                histtype="step",
                lw=1.5,
                alpha=0.99,
                label=label,
            )
    axs[1].legend(loc="r", ncols=1, framealpha=0.0)
    
    # Add labels for each linac sequence.
    sequence_names = [
        "MEBT", 
        "DTL1", 
        "DTL2", 
        "DTL3", 
        "DTL4", 
        "DTL5", 
        "DTL6",
    ]
    sequences = []
    for sequence_name in sequence_names:
        if not any((sequence_name in node) for (node, _, _) in nodes_list):
            continue
        start = nodes_dict[f"{sequence_name}_START"][0]
        stop = nodes_dict[f"{sequence_name}_END"][0]
        sequences.append([start, stop, sequence_name])
            
    cmap = pplt.Colormap("mono")
    _colors = [cmap(i) for i in np.linspace(0.15, 0.8, len(sequences))]
    for i, (start, stop, label) in enumerate(sequences):
        mid = 0.5 * (start + stop)
        axs[0].plot([start, stop], [0.0, 0.0], color=_colors[i], lw=5)
        axs[0].annotate(
            label,
            xy=(mid, 0.5),
            horizontalalignment="center",
            color=_colors[i],
            fontweight="normal",
            fontsize="med-small",
        )
    
    axs[0].format(xspineloc="neither", yspineloc="neither", ylim=(-1.0, 1.0))
    axs.format(xmin=smin, xmax=smax, xlabel="Position [m]")
    if log:
        axs[1:].format(yscale="log", yformatter=None)
    save_figure(f"loss_position_hist_rz_seq_{log}")
    plt.show()

### Location in initial bunch 

Plot the phase space projections of the initial bunch with lost particles highlighted.

#### 2D projections

In [ ]:
n_lost_max = max(lostbunches[label].shape[0] for label in labels)

@interact(
    frame=widgets.BoundedIntText(min=0, max=n_frames-1, value=0),
    dim1=widgets.Dropdown(options=dims, value=dims[-2]),
    dim2=widgets.Dropdown(options=dims, value=dims[-1]),
    normalize=False,
    mask=True,
    log=True,
    bins=widgets.BoundedIntText(min=32, max=200, value=64, continuous_update=False),
    size=widgets.FloatSlider(min=1.0, max=15.0, value=4.0, continuous_update=False),
    nplot=widgets.IntSlider(
        min=0, max=n_lost_max, continuous_update=False, value=n_lost_max
    ),
    plot_xy=True,
    plot_z=True,
)
def update(frame, dim1, dim2, normalize, mask, log, bins, size, nplot, plot_xy, plot_z):
    if dim1 == dim2:
        return
    axis = [dims.index(dim) for dim in [dim1, dim2]]
    
    data = [bunches[label][frame].loc[:, [dim1, dim2]].values for label in labels]
    if normalize:
        data = [ps.cloud.norm_xxp_yyp_zzp(X, scale_emittance=True) for X in data]
    limits = psv.combine_limits([psv.cloud.auto_limits(X, pad=0.0, zero_center=False) for X in data])

    fig, axs = pplt.subplots(ncols=2)
    for ax, label, X in zip(axs, labels, data):
        psv.cloud.plot2d(
            X,
            ax=ax,
            bins=bins,
            limits=limits,
            process_kws=dict(norm="max"),
            offset=(0.0 if mask else 1.0),
            mask=mask,
            vmax=1.0,
            cmap=pplt.Colormap("mono", left=0.04, right=0.9),
            colorbar=True,
            colorbar_kw=dict(tickminor=True, width="1.2em"),
            norm=("log" if log else None),
        )
        bunch = bunches[label][frame]
        if plot_xy:
            if frame == 0:
                idx = lost_index[label]["transverse"]
            else:
                idx = []
                for i in lost_index[label]["transverse"]:
                    _index = bunch[bunch["index"] == i].index
                    if len(_index) == 1:
                        idx.append(int(_index[0]))                
            idx = idx[:nplot]
            psv.cloud.plot2d(
                X[idx, :],
                ax=ax,
                kind="scatter",
                c="pink",
                s=size,
                label="lost_xy",
            )
        if plot_z:
            if frame == 0:
                idx = lost_index[label]["longitudinal"]
            else:
                idx = []
                for i in lost_index[label]["longitudinal"]:
                    _index = bunch[bunch["index"] == i].index
                    if len(_index) == 1:
                        idx.append(int(_index[0]))                
            idx = idx[:nplot]
            psv.cloud.plot2d(
                X[idx, :],
                ax=ax,
                kind="scatter",
                c="green",
                s=size,
                label="lost_z",
            )
    axs.format(xlabel=dims[axis[0]], ylabel=dims[axis[1]])
    axs.format(xlim=limits[0], ylim=limits[1], toplabels=labels)
    axs[1].legend(loc="r", ncols=1, handlelength=1.0)
    plt.show()

#### Corner plot 

In [ ]:
if save:
    for label in labels:
        print(label)

        grid = psv.CornerGrid(d=6, corner=True, labels=dims_units, diag_rspine=True, space=1.4, diag_share=False)

        X = bunches[label][0].loc[:, dims].values
        grid.plot_cloud(
            X,
            bins=85,
            norm="log",
            cmap=pplt.Colormap("mono", left=0.04, right=0.7),
        )    
        for color, key in zip(["pink", "green"], ["transverse", "longitudinal"]):
            grid.plot_cloud(
                X[lost_index[label][key], :],
                kind="scatter",
                diag=False,
                c=color,
                s=1.0,
            )
        grid.format_diag(yscale="log", yformatter="log", ymin=(10.0 ** -6.0))

        ## Add ylabel to upper-left histogram.
        ## Does not work if yspine is on the right! Fix this in CornerGrid.
        # grid.axs[0, 0].format(ylabel="x [mm]") 

        save_figure(f"loss_initbunch_corner_{label}")
        plt.show()

#### Radial distribution [INT]

The following plot is unnormalized.

In [ ]:
_lostbunches = []
for label in labels:
    bunch = bunches[label][0]
    X = bunch.loc[:, dims].values

    Sigma = np.cov(X.T)
    (eps_x, eps_y, eps_z) = ps.ap.apparent_emittance(Sigma)
    V = ps.ap.norm_matrix(*ps.ap.twiss(Sigma))
    A = np.sqrt(np.diag([eps_x, eps_x, eps_y, eps_y, eps_z, eps_z]))

    index = lost_index[label]["transverse"]
    X_lost = ps.cloud.transform_linear(X[index, :], np.linalg.inv(np.matmul(V, A)))
    _lostbunches.append(X_lost)


@interact(
    bins=(10, 30),
    autobin=False,
    log=False,
    density=True,
    x=True,
    px=True,
    y=False,
    py=False,
    z=False,
    pz=False,
)
def update(bins, autobin, log, density, x, px, y, py, z, pz):
    checks = [x, px, y, py, z, pz]
    axis = tuple([i for i, check in enumerate(checks) if check])
    bins = "auto" if autobin else bins
    ylabel = "Density (unnormalized)" if density else "Count"

    fig, ax = pplt.subplots()
    for label, X in zip(labels, _lostbunches):
        radii = ps.cloud.get_radii(X[:, axis])
        ax.hist(radii, bins=bins, density=density, range=(0.0, np.max(radii)), histtype="step", label=label, lw=1.5)
    if log:
        ax.format(yscale="log", yformatter="log")

    ax.legend(loc="r", ncols=1, framealpha=0.0)
        
    _dims = ["x", "x'", "y", "y'", "z", "z'"]
    xlabel = "-".join([_dims[k] for k in axis])
    xlabel = f"Radius ({xlabel})"
    ax.format(ylabel=ylabel, xlabel=xlabel, title="Lost particles in initial bunch")
    plt.show()

#### Radial distribution [SAVE]

In [ ]:
if save:
    for axis in [
        (0, 1),
        (2, 3),
        (4, 5),
        (0, 1, 2, 3),
        (0, 1, 4, 5),
        (2, 3, 4, 5),
        (0, 1, 2, 3, 4, 5),
    ]:
        fig, ax = pplt.subplots()
        for label, _X in zip(labels, _lostbunches):
            radii = ps.cloud.get_radii(_X[:, axis])
            ax.hist(radii, bins=20, density=True, range=(0.0, np.max(radii)), histtype="step", label=label, lw=1.5)
    
        _dims = ["x", "x'", "y", "y'", "z", "z'"]
        xlabel = "-".join([_dims[k] for k in axis])
        xlabel = f"Radius ({xlabel})"
        ax.format(ylabel="Density (unnormalized)", xlabel=xlabel, title="Lost particles in initial bunch")
        ax.legend(loc="upper right", ncols=1)
    
        tag = "-".join([str(k) for k in axis])
        save_figure(f"loss_initbunch_radial_{tag}")
        plt.close()